In [25]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from tqdm import tqdm
import pandas as pd
import time
import datetime


def yogiyo_crawling(location, food_category):
    
    # url 가져오기
    urls = pd.read_csv("./"+location.split(" ")[1]+"_"+food_category+"_url_list.csv")
    
    # 유효한 가게(리뷰개수 10개 이상)만 가져오기 
    is_valid = urls['개수충족'] == 1
    urls = urls[is_valid]
    urls = urls['url'].values.tolist()
    
    address = location
    name = location.split(" ")[1]
    list_url = "https://www.yogiyo.co.kr/mobile/#/"
    
    # 결과 데이터셋 생성
    df = pd.DataFrame(columns=['시', '구', '업종명', '가게명', '년', '월', '전체평점', '맛 평점', '양 평점', '배달 평점', '리뷰 내용', '주문 내역'])

    try:
        # 가게별로 접속하여 리뷰 수집
        for url in tqdm(urls):
            # 해당 가게 url로 접속
            driver = webdriver.Chrome('/Users/jijoonghong/Downloads/chromedriver')
            driver.get(url)
            time.sleep(3)
            
            # 리뷰 개수
            num_of_review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]')
            
            # 가게명
            store_name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text
            
            # 리뷰를 볼 수 있는 버튼 찾고 클릭  
            try:
                review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            except:
                time.sleep(3)
                review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')

            driver.execute_script("arguments[0].click();", review)
            time.sleep(1)

            # 전체 리뷰 개수 확인    
            try:
                num_of_review = driver.find_element_by_xpath(
                    '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]')
                if num_of_review == '':
                    num_of_review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span')
            except:
                time.sleep(3)
                num_of_review = driver.find_element_by_xpath(
                    '//*[@id="content"]/div[2]/div[1]/div[5]/div[2]/div/strong[1]')
                if num_of_review == '':
                    num_of_review = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span')

                    
            # 동적으로 변하는 더보기 버튼의 인덱스 계산(더보기 클릭 시 10개씩 증가) 및 전체 리뷰 확인 가능할 때 까지 클릭      
            j = 0
            idx = 11
            max_idx = int(num_of_review.text) // 10

            while True:
                if j == max_idx:
                    html = driver.page_source
                    break
                try:
                    driver.find_element_by_css_selector('#review > li.list-group-item.btn-more > a').click()
                    time.sleep(1.5)

                    dates = driver.find_element_by_xpath(
                        "// *[ @ id = 'review'] / li[" + str(idx) + "] / div[1] / span[2]").text
                    if "전" in dates or "어제" in dates or int(dates.split("년")[0]) > 2019:
                        pass
                    elif int(dates.split("년")[0]) == 2019 and int(dates.split(" ")[1][:-1]) >= 3:
                        pass
                    else:
                        html = driver.page_source
                        break

                except Exception as e: # 네트워크 지연에 따른 예외 발생 시 대기 후 재실행
                    print(e)
                    time.sleep(3)
                    driver.find_element_by_css_selector('#review > li.list-group-item.btn-more > a').click()
                    time.sleep(1.5)

                    dates = driver.find_element_by_xpath(
                        "// *[ @ id = 'review'] / li[" + str(idx) + "] / div[1] / span[2]").text
                    if "전" in dates or "어제" in dates or int(dates.split("년")[0]) > 2019:
                        pass
                    elif int(dates.split("년")[0]) == 2019 and int(dates.split(" ")[1][:-1]) >= 3:
                        pass
                    else:
                        html = driver.page_source
                        break
                j += 1
                idx += 10

            # 전체 리뷰 확보 후 html 파싱    
            soup = BeautifulSoup(html, 'html.parser')
            
            # 리뷰만 가져오기 
            reviews = soup.find_all(class_='list-group-item star-point ng-scope')
            
            # 데이터 삽입 전 전처리
            for review in reviews:
                # 날짜 계산
                if "전" in review.find(class_="review-time ng-binding").get_text() or (
                        "어제" in review.find(class_="review-time ng-binding").get_text()):
                    year = 2021
                    month = datetime.date.today().month
                else:
                    year = int(review.find(class_="review-time ng-binding").get_text().split("년")[0])
                    month = int(review.find(class_="review-time ng-binding").get_text().split(" ")[1][0])

                # 리뷰     
                comment = review.find('p', attrs={'ng-show': 'review.comment'})
                comment = comment.get_text().replace("\n", " ").replace("  ", " ")
                
                # 주문 메뉴 
                menu = review.find('div', attrs={'class': "order-items default ng-binding"}).get_text()
                
                # 전체 평점 
                overall = review.find_all(class_="full ng-scope")
                
                # 세부 평점 
                points = review.find_all(class_="points ng-binding")
                try:
                    taste = int(points[0].get_text()) # 맛평점 
                except:
                    taste = ""
                try:
                    quantity = int(points[1].get_text()) # 양평점 
                except:
                    quantity = ""
                try:
                    delivery = int(points[2].get_text()) # 배달 평점
                except:
                    delivery = ""

                # 하나의 리뷰 데이터 구성     
                row = pd.DataFrame(
                    [(address.split(" ")[0], address.split(" ")[1], food_category, store_name, year, month,
                      len(overall), taste, quantity, delivery, comment, menu)],
                    columns=['시', '구', '업종명', '가게명', '년', '월', '전체평점', '맛 평점', '양 평점', '배달 평점', '리뷰 내용', '주문 내역'])
                
                # 최종 데이터셋에 추가
                df = df.append(row)

            driver.close()

    # 지나치게 많은 리뷰와 네트워크 지연 등의 문제로 동적 페이지 크롤링의 오류 다수 증가, 종료된 위치와 중간 결과를 저장함으로써 대처        
    except Exception as e:
        name = location.split(" ")[1]
        df_name = "{}_{}".format(name, food_category)
        df.to_csv("./{}_비정상종료.csv".format(df_name), encoding='utf-8-sig')
        print("종료위치 {} - {} - {}".format(food_category, urls.index(url), idx))
        print(e)


    df_name = "{}_{}".format(name, food_category)
    df.to_csv("./{}.csv".format(df_name), encoding='utf-8-sig')



In [28]:
def main():
    categories = ["1인분주문","프랜차이즈", "치킨", "피자양식", "중국집", "한식", "일식돈까스", "족발보쌈", "야식", "분식", "카페디저트"]
    final_start = datetime.datetime.now()

    for category in categories:
        start = datetime.datetime.now()
        yogiyo_crawling('서울특별시 강남구 삼성동 16-1 강남구청', category)
        end = datetime.datetime.now()
        t = end - start
        hours, remainder = divmod(t.seconds, 3600)
        print("{} : {}시간 {}분".format(category, hours, remainder))

    final_end = datetime.datetime.now()

    t = final_end - final_start
    hours, remainder = divmod(t.seconds, 3600)
    print(final_start)
    print(final_end)
    print("전체소요시간 : {}시간 {}분".format(hours, remainder))

In [30]:
main()

  0%|          | 0/28 [00:00<?, ?it/s]

55


  4%|▎         | 1/28 [00:16<07:36, 16.92s/it]

53


  7%|▋         | 2/28 [00:34<07:23, 17.06s/it]

...

11


 96%|█████████▋| 27/28 [05:42<00:10, 10.11s/it]

11


100%|██████████| 28/28 [05:52<00:00, 12.60s/it]

야식 : 0시간 352분
2021-08-04 01:25:35.536116
2021-08-04 01:31:28.233600
전체소요시간 : 0시간 352분
